In [1]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.figure_factory as ff
import numpy as np

In [2]:
def concatenate_pickles(directory_path):
    # Get a list of all .pkl files in the given directory
    pkl_files = [f for f in os.listdir(directory_path) if f.endswith('.pkl')]

    # Initialize an empty list to store individual DataFrames
    dfs = []

    # Load each pickle file and append to dfs list
    for pkl_file in pkl_files:
        full_path = os.path.join(directory_path, pkl_file)
        dfs.append(pd.read_pickle(full_path))

    # Concatenate all DataFrames into a single DataFrame
    final_df = pd.concat(dfs, ignore_index=True)
    
    final_df = final_df.sort_values(by=['Fall Type', 'model', 'Angle']).reset_index(drop=True)

    return final_df

In [3]:
directory_path = './Data'
final_df = concatenate_pickles(directory_path)

In [4]:
final_df = final_df.drop_duplicates(subset=['Angle','Fall Type','model'])

In [8]:
# Pivot tables based on 'model' and 'Fall Type'
color_table = final_df.pivot_table(index='model', columns='Fall Type', values='Fall Detection', aggfunc='sum')

# Rearrange rows in the specified order
desired_order = ['yolov8n-pose.pt', 'yolov8s-pose.pt', 'yolov8m-pose.pt']
color_table = color_table.reindex(desired_order)

# Create heatmap with black text color
fig = ff.create_annotated_heatmap(
    z=color_table.values,
    x=color_table.columns.tolist(),
    y=color_table.index.tolist(),
    annotation_text=color_table.fillna('').values.astype(str),  # Using color_table here
    colorscale='RdYlGn',
    font_colors=['black'],
    showscale=True
)

# Update layout to add labels
fig.update_layout(
    title="Model vs Fall Type",
    xaxis_title="Fall Type",
    yaxis_title="Model"
)

# Add label to the colorbar
fig['data'][0]['colorbar']['title'] = 'Number of Falls'
fig.show()

In [10]:
# Pivot tables based on 'model' and 'Angle'
color_table_angle = final_df.pivot_table(index='model', columns='Angle', values='Fall Detection', aggfunc='sum')

# Rearrange rows in the specified order
desired_order = ['yolov8n-pose.pt', 'yolov8s-pose.pt', 'yolov8m-pose.pt']
color_table_angle = color_table_angle.reindex(desired_order)

# Get the min and max values from the color_table for the colorbar
min_val = color_table_angle.min().min()
max_val = color_table_angle.max().max()

# Create evenly spaced tick values between min and max
tick_values = np.linspace(min_val, max_val, 5)  # Adjust the number 5 to change the number of ticks

# Create heatmap with black text color
fig_angle = ff.create_annotated_heatmap(
    z=color_table_angle.values,
    x=color_table_angle.columns.tolist(),
    y=color_table_angle.index.tolist(),
    annotation_text=color_table_angle.fillna('').values.astype(str),
    colorscale='RdYlGn',
    font_colors=['black'],
    showscale=True,
    colorbar=dict(
        tickvals=tick_values,
        ticktext=[str(int(val)) for val in tick_values],
        title='Number of Falls'
    )
)

# Update layout to add labels
fig_angle.update_layout(
    title="Model vs Angle",
    xaxis_title="Angle",
    yaxis_title="Model"
)

fig_angle.show()

In [11]:
color_table_ft_angle = final_df.pivot_table(index='Fall Type', columns='Angle', values='Fall Detection', aggfunc='sum')

# Get the min and max values from the color_table for the colorbar
min_val = color_table_ft_angle.min().min()
max_val = color_table_ft_angle.max().max()

# Create evenly spaced tick values between min and max as whole numbers
tick_values = np.linspace(min_val, max_val, 5)  # Adjust the number 5 to change the number of ticks
tick_values = [int(val) for val in tick_values]

# Create heatmap with black text color
fig_ft_angle = ff.create_annotated_heatmap(
    z=color_table_ft_angle.values,
    x=color_table_ft_angle.columns.tolist(),
    y=color_table_ft_angle.index.tolist(),
    annotation_text=color_table_ft_angle.fillna('').values.astype(str),  # Using color_table_ft_angle here
    colorscale='RdYlGn',
    font_colors=['black'],
    showscale=True,
    colorbar=dict(
        tickvals=tick_values,
        ticktext=tick_values,
        title='Number of Falls'
    )
)

# Update layout to add labels
fig_ft_angle.update_layout(
    title="Fall Type vs Angle",
    xaxis_title="Angle",
    yaxis_title="Fall Type"
)

fig_ft_angle.show()
